In [ ]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import pickle
from sklearn.decomposition import IncrementalPCA
from speculator import SpectrumPCA
from speculator import Speculator
dtype = tf.float32

### PCA basis consutrction

Create PCA compression object

In [ ]:
PCABasis = SpectrumPCA(n_parameters = , # number of parameters
                       n_wavelengths = , # number of wavelength values
                       n_pcas = , # number of pca coefficients to include in the basis
                       log_spectrum_filenames = , # list of filenames containing the (un-normalized) log spectra for training the PCA
                       parameter_filenames = , # list of filenames containing the corresponding parameter values
                       parameter_selection = None) # pass an optional function that takes in parameter vector(s) and returns True/False for any extra parameter cuts we want to impose on the training sample (eg we may want to restrict the parameter ranges)

Perform the PCA compression in the following steps:<br/>
(1) compute shift and scale for both parameters and (log) spectra (looping over the data files)<br/>
(2) compute (incremental) PCA basis for the training data<br/>
(3) transform the training data into the PCA basis for subsequent training with neural network<br/>
(4) transform a validation set into the PCA basis to interrogate the accuracy of the basis set

In [ ]:
PCABasis.compute_spectrum_parameters_shift_and_scale() # computes shifts and scales for (log) spectra and parameters

PCABasis.train_pca()

PCABasis.transform_and_stack_training_data(filename = , # filename = path + prefix for saving stacked training data (both parameters and pca coefficients files)
                                           retain = True) # retain = True will keep stacked versions of the training data (both parameters and pca coefficients) as class attributes

validation_spectra, validation_spectra_in_pca_basis = PCABasis.validate_pca_basis(log_spectrum_filename = # spectrum filename is a file of validation (log) spectra to test the PCA basis out on
                                                                                 ) 

### Emulator

Import the training data (parameters and pca coefficients; note that shifts and re-scaling for all inputs/outputs are taken care of internally to the Speculator object so you do not need to shift/scale the training data at this stage)

In [ ]:
training_theta = tf.convert_to_tensor(PCABasis.training_parameters.astype(np.float32))
training_pca = tf.convert_to_tensor(PCABasis.training_pca.astype(np.float32))

Initialize the PCA neural network emulator

In [ ]:
speculator = Speculator(n_parameters = , # number of model parameters 
                       wavelengths = , # array of wavelengths
                       pca_transform_matrix = PCABasis.pca_transform_matrix, 
                       parameters_shift = PCABasis.parameter_shift, 
                       parameters_scale = PCABasis.parameter_scale, 
                       pca_shift = PCABasis.pca_shift, 
                       pca_scale = PCABasis.pca_scale, 
                       log_spectrum_shift = PCABasis.log_spectrum_shift, 
                       log_spectrum_scale = PCABasis.log_spectrum_scale, 
                       n_hidden = [256, 256, 256], # network architecture (list of hidden units per layer)
                       restore = False, 
                       restore_filename = # path + filename prefix for saving the model
                       optimizer = tf.keras.optimizers.Adam()) # optimizer for model training

Train the emulator: example utilizing a cooling schedule with respect to both batch size (increasing) and learning rate (decreasing), and doing gradient accumulation for very large batch sizes (to avoid running out of memory when training on a GPU)

In [ ]:
# cooling schedule
validation_split = 0.1
lr = [1e-3, 1e-4, 1e-5, 1e-6]
batch_size = [1000, 10000, 50000, int((1-validation_split) * training_theta.shape[0])]
gradient_accumulation_steps = [1, 1, 1, 10] # split the largest batch size into 10 when computing gradients to avoid memory overflow

# early stopping set up
patience = 20
max_epochs = 1000

# train using cooling/heating schedule for lr/batch-size
for i in range(len(lr)):

    print('learning rate = ' + str(lr[i]) + ', batch size = ' + str(batch_size[i]))

    # set learning rate
    speculator.optimizer.lr = lr[i]

    # split into validation and training sub-sets
    n_validation = int(training_theta.shape[0] * validation_split)
    n_training = training_theta.shape[0] - n_validation
    training_selection = tf.random.shuffle([True] * n_training + [False] * n_validation)

    # create iterable dataset (given batch size)
    training_data = tf.data.Dataset.from_tensor_slices((training_theta[training_selection], training_pca[training_selection])).shuffle(n_training).batch(batch_size[i])

    # set up training loss
    training_loss = [np.infty]
    validation_loss = [np.infty]
    best_loss = np.infty
    early_stopping_counter = 0

    # loop over epochs
    with trange(max_epochs) as t:
        
        for epoch in t:

            # loop over batches
            for theta, pca in training_data:

                # training step: check whether to accumulate gradients or not (only worth doing this for very large batch sizes)
                if gradient_accumulation_steps[i] == 1:
                    loss = speculator.training_step_pca(pca, theta)
                else:
                    loss = speculator.training_step_with_accumulated_gradients_pca(pca, theta, accumulation_steps=gradient_accumulation_steps[i])

            # compute validation loss at the end of the epoch
            validation_loss.append(speculator.compute_loss_pca(training_pca[~training_selection], training_theta[~training_selection]).numpy())

            # update the progressbar
            t.set_postfix(loss=validation_loss[-1])
            
            # early stopping condition
            if validation_loss[-1] < best_loss:
                best_loss = validation_loss[-1]
                early_stopping_counter = 0
            else:
                early_stopping_counter += 1
            if early_stopping_counter >= patience:
                speculator.update_emulator_parameters()
                speculator.save("model")
                print('Validation loss = ' + str(best_loss))
                break

Load in the trained model and call it

In [ ]:
speculator = Speculator(restore=True, 
                        restore_filename= # path + prefix of the saved model file
                       )

theta = # chose some input parameters to evaluate your (log) spectrum for
log_spectrum = speculator.log_spectrum_(theta) # compute log spectrum

plt.plot(speculator.wavelengths, log_spectrum)
plt.show()